In [2]:
import torch
from torchtext import data, datasets
import random
import numpy as np

SEED = 1111

In [38]:
TEXT = data.Field(tokenize='spacy', batch_first=True)
LABEL = data.LabelField(dtype=torch.float)

train, test = datasets.IMDB.splits(TEXT, LABEL)

train, valid = train.split(random_state=random.seed(SEED))

In [39]:
MAX_VOCAB_SIZE = 20000

TEXT.build_vocab(train,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = 'glove.6B.100d',
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train)

In [40]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = BATCH_SIZE,
    device=device)

In [86]:
import torch.nn as nn
import torch.nn.functional as F

In [72]:
class CNN1d(nn.Module):
  def __init__(self, vocab_size, embedding_dim, n_filters, 
               filter_size, output_dim, dropout, pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
    self.convs = nn.ModuleList([nn.Conv1d(in_channels = embedding_dim, 
                                          out_channels = n_filters,
                                          kernel_size = fs) for fs in filter_size])
    self.linear = nn.Linear(len(filter_size) * n_filters, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, text):
    embedded = self.embedding(text)
    embedded = embedded.permute(0, 2, 1)
    convd = [F.relu(conv(embedded)) for conv in self.convs]
    pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in convd]

    cat = self.dropout(torch.cat(pooled, dim=1))

    return self.linear(cat)


In [73]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZE = [3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZE, OUTPUT_DIM, DROPOUT, PAD_IDX )

In [74]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.5928, -1.6730, -1.4509,  ...,  0.7008,  0.9171, -1.0891],
        [-0.1248, -0.0679, -0.1765,  ...,  1.1187, -1.8712, -0.4191],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.2715,  0.9957,  1.1696,  ...,  0.7124,  0.2630, -0.5472],
        [-0.2365,  0.2716,  0.1552,  ..., -0.5834,  0.3832,  0.2393],
        [ 0.1881,  0.3164,  0.6914,  ..., -0.5092,  0.7231,  0.6136]])

In [75]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [76]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [77]:
def accuracy(predictions, y):
  pred = torch.round(torch.sigmoid(predictions))
  actual = (pred == y).float()
  acc = actual.sum() / len(actual)
  return acc

In [54]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.train()
  for batch in iterator:
    optimizer.zero_grad()
    predictions = model(batch.text).squeeze(1)
    loss = criterion(predictions, batch.label)
    acc = accuracy(predictions, batch.label)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [57]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.eval()

  with torch.no_grad():
    for batch in iterator:
      predictions = model(batch.text).squeeze(1)
      loss = criterion(predictions, batch.label)
      acc = accuracy(predictions, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [79]:
import time
N_EPOCHS = 5

opt_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss, train_acc = train(model, train_iter, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
  end_time = time.time()

  if valid_loss < opt_valid_loss:
    opt_valid_loss = opt_valid_loss
    torch.save(model.state_dict(), 'CNN1d_RNN_TC.pt')

  print(f'Epoc: {epoch+1:02}')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Accuracy: {train_acc:.3f}')
  print(f'\tValidation Loss: {valid_loss:.3f} | Validation Accuracy: {valid_acc:.3f}')

Epoc: 01
	Train Loss: 0.027 | Train Accuracy: 0.992
	Validation Loss: 0.470 | Validation Accuracy: 0.870
Epoc: 02
	Train Loss: 0.023 | Train Accuracy: 0.993
	Validation Loss: 0.495 | Validation Accuracy: 0.872
Epoc: 03
	Train Loss: 0.019 | Train Accuracy: 0.995
	Validation Loss: 0.510 | Validation Accuracy: 0.872
Epoc: 04
	Train Loss: 0.015 | Train Accuracy: 0.996
	Validation Loss: 0.537 | Validation Accuracy: 0.876
Epoc: 05
	Train Loss: 0.014 | Train Accuracy: 0.996
	Validation Loss: 0.557 | Validation Accuracy: 0.872


In [81]:
model.load_state_dict(torch.load('CNN1d_RNN_TC.pt'))
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Accuracy: {test_acc:.2f}')

Test Loss: 0.693 | Test Accuracy: 0.85


In [82]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, text):
  model.eval()
  tokenized = [token.text for token in nlp.tokenizer(text)]
  tokenized += ['<pad>'] * len(tokenized)
  index = [TEXT.vocab.stoi[t] for t in tokenized]
  tensor = torch.LongTensor(index).to(device)
  tensor = tensor.unsqueeze(0)
  prediction = torch.sigmoid(model(tensor))
  return prediction.item()

In [83]:
predict_sentiment(model, "The weather is fine")

0.22681736946105957

In [84]:
predict_sentiment(model, "The weather is bad")

0.9999772310256958

In [85]:
predict_sentiment(model, "The weather is not good")

0.3321540951728821